In [56]:
name="小王"
month="三月"
print("{}在{}的工资为{}".format(name, month,5000))

小王在三月的工资为5000


In [61]:

import torch.nn.functional as F
def load_model(model_path, device=None):
    if device is None:
        device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model = torch.load(model_path, map_location=device)
    return model.to(device)
def prepare_input(device, batch_size=1, channels=3, height=640, width=640):
    x = torch.randn(batch_size, channels, height, width, device=device)
    return x
def infer(model_path):
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    
    # 加载模型
    model = load_model(model_path, device)
    model.eval()  # 设置为评估模式
    
    # 准备输入数据
    x = prepare_input(device)
    
    # 执行前向传播
    with torch.no_grad():
        output = model(x)
    
    # 打印输出的形状（作为示例）
    print(output.size())

# 调用推理函数
model_path = 'best.pt'  # 确保这是你的模型文件的正确路径
infer(model_path)


AttributeError: 'dict' object has no attribute 'to'

In [1]:
import cv2

for i in range(100):
    cap = cv2.VideoCapture(i)
    if cap.isOpened():
        print('Webcam found at index: ' + str(i))
        cap.release()

Webcam found at index: 0
Webcam found at index: 1
Webcam found at index: 2


In [ ]:
import cv2
 
video = cv2.VideoCapture("rtmp://localhost:1935/live/123")
fps = video.get(cv2.CAP_PROP_FPS)
print(fps)
size = (int(video.get(cv2.CAP_PROP_FRAME_WIDTH)), int(video.get(cv2.CAP_PROP_FRAME_HEIGHT)))
print(size)
while True:
    ret, frame = video.read()
    cv2.imshow("A video", frame)
    c = cv2.waitKey(1)
    if c == 27:
        break
video.release()
cv2.destroyAllWindows()

In [ ]:
import face_recognition

# 加载第一张照片并识别面部特征
image1 = face_recognition.load_image_file("C:/Users/wsh/Pictures/Camera Roll/f1.jpg")
encoding1 = face_recognition.face_encodings(image1)[0]

# 加载第二张照片并识别面部特征
image2 = face_recognition.load_image_file("C:/Users/wsh/Pictures/Camera Roll/f2.jpg")
encoding2 = face_recognition.face_encodings(image2)[0]

# 比较两个面部特征
results = face_recognition.compare_faces([encoding1], encoding2)

if results[0]:
    print("这两张照片是同一个人。")
else:
    print("这两张照片不是同一个人。")

In [7]:
import time
from pyexpat import model
from typing import List, Any
import asyncio
import aiomysql
import cv2
import jwt
import numpy as np
import torch
from starlette.responses import JSONResponse

from MysqlUtils.init import register_mysql
from RedisUtils.init import register_redis
from SmokingDecisionMaker import SmokingDecisionMaker
from configList import *
from importYoloPt import get_model
from utils.ConfigReader import ConfigReader
from utils.augmentations import letterbox
from utils.general import check_img_size, non_max_suppression, scale_coords
from utils.plots import Annotator
WEIGHTS = 'weights/yolov5n.pt'
IMGSZ = [640, 640]  # 图像尺寸
CONF_THRES = 0.5  # 置信度阈值
IOU_THRES = 0.2  # IOU阈值
MAX_DET = 1000  # 最大检测数量
LINE_THICKNESS = 1  # 线条厚度
HIDE_CONF = False  # 是否隐藏置信度
HIDE_LABELS = None  # 是否隐藏标签
model, device, half, stride, names = get_model()
imgsz = check_img_size([640, 640], s=stride)
def colors(index, bright=True):
    color_list = [(0, 0, 255), (0, 255, 0), (255, 0, 0)]
    return color_list[index % len(color_list)]

def pred_img_optimized_async(img0, model, device, imgsz, names, conf_thres, iou_thres, half,
                                   line_thickness, hide_labels, hide_conf, max_det):
        img = letterbox(img0, new_shape=imgsz, auto=True)[0]
        img = img.transpose((2, 0, 1))[::-1]
        img = np.ascontiguousarray(img)

        img = torch.from_numpy(img).to(device)
        img = img.half() if half else img.float()
        img /= 255.0  # 归一化
        if img.ndimension() == 3:
            img = img.unsqueeze(0)

        im0 = img0.copy()
        annotator = Annotator(im0, line_width=line_thickness, example=str(names))

        pred = model(img, augment=False, visualize=False)[0]
        pred = non_max_suppression(pred, conf_thres, iou_thres, classes=None, agnostic=False, max_det=max_det)

        detections = []
        det = pred[0]
        if len(det):
            det[:, :4] = scale_coords(img.shape[2:], det[:, :4], im0.shape).round()
            for *xyxy, conf, cls in reversed(det):
                c = int(cls)
                detections.append({'coords': tuple(map(int, xyxy)), 'confidence': conf.item()})

                label = None if hide_labels else f'{names[c]} {conf:.2f}' if not hide_conf else names[c]
                annotator.box_label(xyxy, label, color=colors(c))

        return im0, detections
cap = cv2.VideoCapture(0)

# 用于计算FPS的变量
frame_count = 0
start_time = time.time()

while True:
    ret, frame = cap.read()
    if not ret:
        break

    # 对当前帧进行处理和预测
    img, detections =pred_img_optimized_async(frame, model, device, IMGSZ, names, CONF_THRES,
                                                               IOU_THRES, half, LINE_THICKNESS, HIDE_LABELS, HIDE_CONF,
                                                               MAX_DET)

    # 显示处理后的图像
    frame_count += 1
    elapsed_time = time.time() - start_time
    fps = frame_count / elapsed_time

    # 将FPS文本写入图像
    cv2.putText(img, f"FPS: {fps:.2f}", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)

    # 显示处理后的图像
    cv2.imshow('Image', img)
    
    # 按 'q' 键退出循环
    if cv2.waitKey(1) == ord('q'):
        break
cap.release()
cv2.destroyAllWindows()


YOLOv5  b437241 torch 2.2.0+cu118 CUDA:0 (NVIDIA GeForce RTX 2050, 4096MiB)



UnpicklingError: invalid load key, '\x08'.